## HW3. Inference of purity, ploidy, and absolute copy number

In this homework we will develop a model for determining purity, ploidy, and absolute copy number using (non-allelic) copy ratio data from the TCGA Pan-cancer atlas.

https://gdc.cancer.gov/about-data/publications/pancanatlas

Our data is provided as a "seg" file containing the segmented copy data from all samples in the TCGA Pan-can atlas broad.mit.edu_PANCAN_Genome_Wide_SNP_6_whitelisted_min1kprobes.seg. It has the following columns

- Sample - Which TCGA sample this segment was found in
- Chromosome, Start, End - the genomic location of the segment
- Num_Probes - The number of exons supporting the segment (the "probe" terminology is a relic of when copy number was called from SNP arrays, which probed sites of genetic variation in the genome). We filtered small segments for this excerise as they are trickier to call.
- Segment_Mean - The $log_2$ copy ratio of the segment.

Our simplistic model will be as follows. Each segment of the genome in a tumor has some number of copies $c_i$ ranging from from 0...9 with equal probability.

$$c_i \sim Cat([\frac{1}{10},...,\frac{1}{10}])$$

Each tumor has an unknown purity $\alpha$ on which we'll assume a uniform prior

$$\alpha \sim Unif(0,1)$$

as well as a ploidy $\tau$ that is the average of the copy number states across all segments (weighted by the number of probes in the segment, $w_i$)

$$\tau = \frac{1}{\sum_{i=1}^N w_i}\sum_{i=1}^N w_ic_i$$

Then each segment is drawn from a normal distribution with a mean $\mu_{c_i}$ determined by the underlying copy number state, purity, and ploidy, and a variance $\sigma^2$ that is the same for all states.

$$x_i \sim \mathcal{N}(\mu_{c_i},\sigma^2)$$
$$\mu_{c_i} = log_2(\frac{\alpha c_i + (1-\alpha)*2}{\alpha \tau + (1-\alpha)*2})$$

Note that the provided segment means are log copy ratios, not copy ratios, so we take the log of the expected copy ratio to parameterize our distribution.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns

## Question 1. Data Exploration

We'll start by exploring the sample "TCGA-CZ-5454-01A-01D-1499-01", extract the segments for this sample from the seg file and create step plot showing segment mean $log_2$ copy ratio on the y-axis, and genomic range of each segment on the x axis. It is convention to consider the X chromosome to be numbered 23 (the Y would be 24, but it is not included in this seg file), and to plot the chromosomes in order by assigning a unique index to each genomic position. If we have position $i$ on chromome $c$, we would consider its genomic position to be

$$i + \sum_{j=1}^{c-1} L_j$$

where $L_j$ is the length of the $jth$ chromosome (i.e. we add on the lengths of all previous chromosomes). The following file gives the lengths of all chromosomes:

In [2]:
C = pd.read_csv('http://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/latest/hg19.chrom.sizes',sep='\t',
           header=None,names=['chr','len'])

C.head()

,chr,len
0,chr1,249250621
1,chr2,243199373
2,chr3,198022430
3,chr4,191154276
4,chr5,180915260


In [3]:
## Load our seg file
S = pd.read_csv('broad.mit.edu_PANCAN_Genome_Wide_SNP_6_whitelisted_min1kprobes.seg',sep='\t')

S.head()

,",Sample,Chromosome,Start,End,Num_Probes,Segment_Mean"
0,"0,TCGA-OR-A5J1-10A-01D-A29K-01,1,3218610.0,247..."
1,"1,TCGA-OR-A5J1-10A-01D-A29K-01,2,484222.0,4575..."
2,"3,TCGA-OR-A5J1-10A-01D-A29K-01,2,45764419.0,16..."
3,"5,TCGA-OR-A5J1-10A-01D-A29K-01,2,167218669.0,2..."
4,"7,TCGA-OR-A5J1-10A-01D-A29K-01,2,220568094.0,2..."


### Question 2. Determining absolute copy number if we already know the purity and ploidy

### (2a) For this same sample as in Q1, let's assume we already know the purity $\alpha=0.75$, and ploidy $\tau=2.1$ (these were the values inferred by ABSOLUTE).  Determine the expected segment copy ratio $\mu_{c_i}$ as a function of the absolute copy number $c_i$, the purity $\alpha$, and the ploidy $\tau$. Then write a function that given an alpha and tau, calculates a vector with $\mu_0,\mu_1,...,\mu_{9}$ (a.k.a., "the comb")

In [4]:
# Returns a vector of length 10 with expected copy ratios
def calc_mu(alpha,tau):
    
    copy_states = np.arange(0,10)
    
    pass

### (2b)  Now re-create your segmentation plot from Q1, but this time add horizonal lines to your plot at every comb tooth $\mu_0,...,\mu_9$. You should see these lines tend to coincide with the segment values.

In [5]:
def comb_plot(x,alpha,tau):
    pass

### (2c) Write down an expression for $P(c_i|x_i,\alpha, \tau,\sigma^2)$. You can leave it in terms of normal pdf functions. Then implement it as a function

In [6]:
# Returns an N x 10 matrix giving the probability each segment belongs to each copy number state
def calc_state_probabilities(x,alpha,tau,sig2):
    pass

### (2d) Using this function, calculate a N x 10 matrix with the probability that each segment belongs to each of the 10 copy number states for our example sample. Assume $\sigma^2=.01$. Plot results as a heatmap.

## Question 3. Inferring purity and ploidy through EM

## (3a). Write down an expression of the log-likelihood of all segments. Again fixing $\sigma^2=.01$, calculate the log-likelihood of a grid of purity and ploidy values $\alpha=0,...,1$, $\tau=1,...,4$ for the previous sample. Make a heatmap of the results.

$$\log P(X|\alpha,\tau,\sigma^2) = ...$$

In [7]:
def log_likelihood(x,alpha,tau,sig2):
    pass

### (3b) To perform the M-step, we'll need to optimize our expected log-likelihood. Implement this function.

$$Q(\theta,\theta_t) = \mathbb{E}_{C|X,\theta_t}[\log P(X,C|\mu,\tau,\sigma^2)]$$

$$=\sum_{i=1}^N \sum_{c_i=0}^9 P(c_i|x_i,\theta_t) \log \mathcal{N}(x_i|c_i,\mu_{c_i},\sigma^2)$$

$$= -\sum_{i=1}^N \sum_{c_i=0}^9 P(c_i|x_i,\theta_t) (\log (\sqrt{\sigma^2})+\frac{(x_i-\mu_{c_i})^2}{2\sigma^2}) + C$$

In [8]:
# State_probabilities are the matrix of $P(c_i|\theta_t)$ values which will be calculated in the E-step
# If you use scipy's normal pdf function, note that it takes sigma as a parameter and not sigma^2!
def Q(x,state_probabilities,alpha,tau,sig2):    
    pass

### (3c) Take the derivative and solve $\frac{\partial Q}{\partial \sigma^2} = 0$ to come up with an M-step update rule for $\sigma^2$ (hint: it is very similar to what we derived for gaussian mixture models, treat $V=\sigma^2$ as a variable rather than $\sigma$). 

### (3d) Now we'll put it all together and use EM to find these purity and ploidy values for ourselves.

- Pick initial values for the purity $\alpha$, ploidy $\tau$, and segment noise $\sigma^2$
- E-step: Calculate the posterior probabilities each segment has a given copy number state as in Q2C
- M-step: Choose new values of your parameters that maximize the expected log likelihood function $Q$
    - $\tau$ can be estimated as the expected copy ratio $\tau = \sum_{i=1}^N \sum_{c_i=0}^9\frac{w_i c_i P(c_i|\theta_t)}{\sum_{i=1}^N {w_i}}$
    - $\alpha$ can't be solved for analytically, but you can find the value that maximizes Q with an optimization library (see below)
    - $\sigma^2$ can be maximized via your rule in Q3C (or with an optimization library)
- Do this iteratively until the parameters and likelihood stop changing values

### Try initalizing with values $\alpha_0=.6$,$\tau_0=2$,$\sigma^2_0=.01$ and $\alpha_0=.2$,$\tau_0=4$,$\sigma^2_0=.01$. Re-do the "comb plot" and state probability heatmaps with your optimized values in each case. How does it do?


In [9]:
# Example usage of scipy minimize
from scipy.optimize import minimize

# Example function to minimize, y=(x-3)^2+2
def function_to_minimize(x):
    return (x-3)**2+2

# We need to provide a starting point for optimization, x0
# Additionally you can provide bounds on the value you are optimizing, if desired. 
##  e.g. here we constrain x to be between 0 and 5
optim_res = minimize(function_to_minimize,x0=5,bounds=[(0,5)])

# This funtion is minimized at x=3
optim_res.x

array([3.])

In [10]:
def EM(x,w,alpha_0,tau_0,sig2_0):
    
    alpha=alpha_0
    tau=tau_0
    sig2=sig2_0

    nit = 20 # Is this enough?
    
    for it in range(0,nit):
        

        # E-step: calculate state probabilities given current parameters from Q2C
        # state_probabilities = ...
        
        # M-step: Update tau, alpha, sigma^2
        # tau = ...
        # alpha = ...
        # sig2 = ...
        
        # Check this is always increasing
        ll = log_likelihood(x,alpha,tau,sig2)
        print(ll)
        
    return(pc,alpha,tau,sig2,ll)        
        
        

### (3e) Try your EM algorithm out on the segmentation profiles of the following samples as well. Pick a range of initial parameter values, and see which converge to a solution with the highest likelihood. Report the purity, ploidy, and plot a heatmap of the copy number state probabilities. See if you get similar solutions to ABSOLUTE, provided in the file TCGA_mastercalls.abs_tables_JSedit.fixed.txt

- TCGA-CW-6090-01A-11D-1668-01
- TCGA-B6-A0RP-01A-21D-A087-01